In [1]:
# Rotten Tomatoes Parser. Will obtain RT score and Audience Score, Director, Writers, and Top Billed Actors/Actresses
from pymongo import MongoClient
import pandas as pd
import numpy as np
import random

# Requests sends and receives HTTP requests
import requests
from bs4 import BeautifulSoup

#used if i need wildcard in bs
import re

import time
import pprint

In [2]:
client = MongoClient('localhost', 27017)
db = client['scraper']
whole_page_ratings = db['whole_page_rt']
rt_ratings = db['rt_ratings']

In [3]:
# Pull in cleaned movie list
budgets_df = pd.read_csv('data/budgets_pd.csv')
movies = budgets_df['title'].tolist()

movies_copy = movies.copy()

cleaned = []
for string in movies_copy:
    clean = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),string)
    cleaned.append(clean)
print(cleaned)

['Avengers: Endgame', 'Pirates of the Caribbean: On Stranger Tides', 'Avengers: Age of Ultron', 'Star Wars Ep. VII: The Force Awakens', 'Avengers: Infinity War', 'Pirates of the Caribbean: At World’s End', 'Justice League', 'Spectre', 'Star Wars: The Rise of Skywalker', 'Solo: A Star Wars Story', 'John Carter', 'Batman v Superman: Dawn of Justice', 'The Lion King', 'Tangled', 'Spider-Man 3', 'Captain America: Civil War', 'Harry Potter and the Half-Blood Prince', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: The Battle of the Five Armies', 'The Fate of the Furious', 'No Time to Die', 'Avatar', 'Superman Returns', 'The Dark Knight Rises', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'Quantum of Solace', 'The Avengers', 'Pirates of the Caribbean: Dead Man’s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Lone Ranger', 'Tenet', 'The Amazing Spider-Man', 'Battleship', 'Transformers: The Last Knight', 'Jurassic World', 'Men in Black 3', 'Transformers: Re

In [139]:
formatted = []
for movie in cleaned:
    _ = movie.replace(': ', '_')
    add = _.replace(' ', '_')
    add = add.replace('-', '_')
    add = add.replace('.', '')
    
    formatted.append(add)
    
print(formatted)

['Avengers_Endgame', 'Pirates_of_the_Caribbean_On_Stranger_Tides', 'Avengers_Age_of_Ultron', 'Star_Wars_Ep_VII_The_Force_Awakens', 'Avengers_Infinity_War', 'Pirates_of_the_Caribbean_At_World’s_End', 'Justice_League', 'Spectre', 'Star_Wars_The_Rise_of_Skywalker', 'Solo_A_Star_Wars_Story', 'John_Carter', 'Batman_v_Superman_Dawn_of_Justice', 'The_Lion_King', 'Tangled', 'Spider_Man_3', 'Captain_America_Civil_War', 'Harry_Potter_and_the_Half_Blood_Prince', 'The_Hobbit_The_Desolation_of_Smaug', 'The_Hobbit_The_Battle_of_the_Five_Armies', 'The_Fate_of_the_Furious', 'No_Time_to_Die', 'Avatar', 'Superman_Returns', 'The_Dark_Knight_Rises', 'Pirates_of_the_Caribbean_Dead_Men_Tell_No_Tales', 'Quantum_of_Solace', 'The_Avengers', 'Pirates_of_the_Caribbean_Dead_Man’s_Chest', 'Man_of_Steel', 'The_Chronicles_of_Narnia_Prince_Caspian', 'The_Lone_Ranger', 'Tenet', 'The_Amazing_Spider_Man', 'Battleship', 'Transformers_The_Last_Knight', 'Jurassic_World', 'Men_in_Black_3', 'Transformers_Revenge_of_the_Falle

In [13]:
# Ex. result URL https://www.rottentomatoes.com/m/avengers_endgame
main_url= 'https://www.rottentomatoes.com/m/'
links = []
for movie in formatted:
    link = f'{main_url}{movie}'
    links.append(link)
print(links[0])

https://www.rottentomatoes.com/m/Avengers_Endgame


In [ ]:
# If request fails make sure to try to grab these movies again
try_again = []

In [106]:
r = requests.get(links[1])
r.status_code
if r.status_code != 200:
    try_again.append(movie)

In [107]:
pprint.pprint(r.text)

('<!DOCTYPE html>\n'
 '<html lang="en"\n'
 '      dir="ltr"\n'
 '      xmlns:fb="http://www.facebook.com/2008/fbml"\n'
 '      xmlns:og="http://opengraphprotocol.org/schema/">\n'
 '\n'
 '    <head prefix="og: http://ogp.me/ns# flixstertomatoes: '
 'http://ogp.me/ns/apps/flixstertomatoes#">\n'
 '\n'
 '        <!-- salt=lay-def-02-juRm -->\n'
 '        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" '
 '/>\n'
 '        <meta http-equiv="x-ua-compatible" content="ie=edge">\n'
 '        <meta name="viewport" content="width=device-width, '
 'initial-scale=1">\n'
 '\n'
 '        <title>Pirates of the Caribbean: On Stranger Tides (2011) - Rotten '
 'Tomatoes</title>\n'
 '        <meta name="description" content="Johnny Depp returns to his iconic '
 'role of Captain Jack Sparrow in an action-packed adventure. Crossing paths '
 "with the enigmatic Angelica (Penélope Cruz), he's not sure if it's love-or "
 "if she's a ruthless con artist who's using him to find the fabled Foun

 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/atlanta/s02/#desktopEpisodeList"> Atlanta: Robbin&#39; Season '
 '</a></li>\n'
 '                                        </div>\n'
 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/fargo/s03/#desktopEpisodeList"> Fargo: Season 3 </a></li>\n'
 '                                        </div>\n'
 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/the_flash/s06/#desktopEpisodeList"> The Flash: Season 6 </a></li>\n'
 '                                        </div>\n'
 '                                    

 '        View All<span class="glyphicon" />\n'
 '    </a>\n'
 '</div>\n'
 '\n'
 '            </section>\n'
 '            <section class="tab-pane fade " id="on-tv">\n'
 '                \n'
 '    <h3>New TV Tonight</h3>\n'
 '    <table class="movie_list tv_list">\n'
 '        \n'
 '            \n'
 '            \n'
 '            <tr class="tv_show_tr tvList1">\n'
 '    <td class="left_col">\n'
 '        <a href="/tv/stargirl/s01">\n'
 '            \n'
 '                <span class="icon tiny certified_fresh"></span>\n'
 '                <span class="tMeterScore">91%</span>\n'
 '            \n'
 '        </a>\n'
 '    </td>\n'
 '    <td class="middle_col">\n'
 '        <a href="/tv/stargirl/s01">\n'
 '            \n'
 '                Stargirl\n'
 '            \n'
 '        </a>\n'
 '    </td>\n'
 '</tr>\n'
 '\n'
 '         \n'
 '            \n'
 '            \n'
 '            <tr class="tv_show_tr tvList1">\n'
 '    <td class="left_col">\n'
 '        <a href="/tv/greenleaf/s05">\n'
 '

 'js-user-rating-error-close-button"></button>\n'
 '                    <h3 class="user-rating-error__header '
 'js-user-rating-error-header"></h3>\n'
 '                    <div class="user-rating-error__body '
 'js-user-rating-error-text"></div>\n'
 '                </section>\n'
 '            </div>\n'
 '\n'
 '            \n'
 '            \n'
 '    \n'
 '    \n'
 '    \n'
 '        \n'
 '    \n'
 '    <aside id="interscroller_ad" class="interscroller_ad visible-xs center '
 'mobile-interscroller " style="width:300px"></aside>\n'
 '    <script>\n'
 '        var mps = mps || {};\n'
 '        mps._queue = mps._queue || {};\n'
 '        mps._queue.gptloaded = mps._queue.gptloaded || [];\n'
 '        mps._queue.adload = mps._queue.adload || [];\n'
 '\n'
 '        mps._queue.gptloaded.push(function() {\n'
 "            if (mps.getResponsiveSet() =='0') { //MOBILE\n"
 "                mps.insertAd('#interscroller_ad','interscroller');\n"
 '            }\n'
 '        });\n'
 '        mps._q

 '        </span>\n'
 '    </div>\n'
 '</div>\n'
 '\n'
 '        \n'
 '            \n'
 '            \n'
 '            <div class="cast-item media inlineBlock  castRemaining">\n'
 '    <a href="/celebrity/brea_berrett" class="pull-left">\n'
 '        \n'
 '            <img '
 'data-src="/assets/pizza-pie/images/poster_default_thumbnail.2ec144e61b4.jpg" '
 'class="js-lazyLoad actorThumb medium media-object" />\n'
 '        \n'
 '    </a>\n'
 '    <div class="media-body">\n'
 '        <a href="/celebrity/brea_berrett" class="unstyled articleLink">\n'
 '            <span title="Brea Berrett">\n'
 '                Brea Berrett\n'
 '            </span>\n'
 '        </a>\n'
 '        <span class="characters subtle smaller" title="Mermaid">\n'
 '            \n'
 '            <br/>as\n'
 '                \n'
 '                    Mermaid\n'
 '                \n'
 '            \n'
 '        </span>\n'
 '    </div>\n'
 '</div>\n'
 '\n'
 '        \n'
 '            \n'
 '            \n'
 '        

 '            \n'
 '        </div>\n'
 '    </div>\n'
 '</aside>\n'
 '\n'
 '\n'
 '        \n'
 '        \n'
 '\n'
 '    <script '
 'src="//player.theplatform.com/pdk/dCK2IC/tpPdkController.js?pdk=5.7.14"></script>\n'
 '    </section>\n'
 '\n'
 '            </div>\n'
 '\n'
 '            \n'
 '    \n'
 '    \n'
 '    <div class="sleaderboard_wrapper hidden-xs mobile-hidden">\n'
 '    <div id="leaderboard_bottom_ad" '
 'style="margin-left:auto;margin-right:auto;display:inline-block"> </div>\n'
 '    <script>\n'
 '        var mps = mps||{}; mps._queue = mps._queue||{}; mps._queue.gptloaded '
 '= mps._queue.gptloaded||[];\n'
 '        mps._queue.gptloaded.push(function () {\n'
 "            if (mps.getResponsiveSet() != '0') { //DESKTOP or TABLET\n"
 "                mps.insertAd('#leaderboard_bottom_ad','bottombanner');\n"
 '            }\n'
 '        });\n'
 '    </script>\n'
 '    </div>\n'
 '\n'
 '\n'
 '\n'
 '            \n'
 '            \n'
 '                \n'
 '    \n'
 '        \n

In [108]:
soup = BeautifulSoup(r.text, 'html.parser')

In [109]:
for rating in soup.find_all('h2', class_="mop-ratings-wrap__score"):
    print(rating.text)





                    33%
                






                    54%
                




In [110]:
# RT rating
rt_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[0].text

# Audience rating
aud_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[1].text

In [111]:
# RT Cleaning
rt_rating = rt_rating.replace("  ", '')
rt_rating = rt_rating.replace("\n", '')
print(rt_rating)

33%


In [112]:
# Audience Cleaning
aud_rating = aud_rating.replace("  ", '')
aud_rating = aud_rating.replace("\n", '')
print(aud_rating)

54%


In [113]:
# Director NEEDS CLEANING
director = soup.find_all('li', class_="meta-row clearfix")[2].text

In [120]:
# Director Cleaned
director = director.replace("  ", '')
director = director.replace("\n", '')
director = director.replace("Directed By:", '')
director = director.split(',')
print(director)

['Anthony Russo']


In [115]:
# Writers NEEDS CLEANING
writers = soup.find_all('li', class_="meta-row clearfix")[3].text

In [116]:
# Writers Cleaned

writers = writers.replace("  ", '')
writers = writers.replace("\n", '')
writers = writers.replace("Written By:", '')
writers = writers.split(',')
print(writers)

['Rob Marshall', ' Terry Rossio', ' Ted Elliott']


In [117]:
# Top 5 billed Actors/Actresses

test_ = soup.find_all('div', class_="castSection")[0].text
test = test_.split("  \n  ")
# print(test)
names = []
for name in test:
    name = name.replace('\nas\n ', '')
    name = name.replace('\n', '')
    name = name.replace('  ', '')
    names.append(name)


act = names[0:13:3]
print(act)

['Johnny Depp ', 'Penelope Cruz ', 'Ian McShane ', 'Geoffrey Rush ', 'Kevin McNally ']


In [121]:
# Set up loop for requests
# Rotten Tomatoes Parser. Will obtain RT score and Audience Score, Director, Writers, and Top Billed Actors/Actresses
from pymongo import MongoClient
import pandas as pd
import numpy as np
import random

# Requests sends and receives HTTP requests
import requests
from bs4 import BeautifulSoup

#used if i need wildcard in bs
import re

import time
import pprint

In [122]:
client = MongoClient('localhost', 27017)
db = client['scraper']
whole_page_ratings = db['whole_page_rt']
rt_ratings = db['rt_ratings']

In [143]:
# Pull in cleaned movie list
budgets_df = pd.read_csv('data/budgets_pd.csv')
movies = budgets_df['title'].tolist()

movies_copy = movies.copy()

cleaned = []
for string in movies_copy:
    clean = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),string)
    cleaned.append(clean)
print(cleaned)

try_again = []

['Avengers: Endgame', 'Pirates of the Caribbean: On Stranger Tides', 'Avengers: Age of Ultron', 'Star Wars Ep. VII: The Force Awakens', 'Avengers: Infinity War', 'Pirates of the Caribbean: At World’s End', 'Justice League', 'Spectre', 'Star Wars: The Rise of Skywalker', 'Solo: A Star Wars Story', 'John Carter', 'Batman v Superman: Dawn of Justice', 'The Lion King', 'Tangled', 'Spider-Man 3', 'Captain America: Civil War', 'Harry Potter and the Half-Blood Prince', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: The Battle of the Five Armies', 'The Fate of the Furious', 'No Time to Die', 'Avatar', 'Superman Returns', 'The Dark Knight Rises', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'Quantum of Solace', 'The Avengers', 'Pirates of the Caribbean: Dead Man’s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Lone Ranger', 'Tenet', 'The Amazing Spider-Man', 'Battleship', 'Transformers: The Last Knight', 'Jurassic World', 'Men in Black 3', 'Transformers: Re

In [144]:
# Requests
# Ex. result URL https://www.rottentomatoes.com/m/avengers_endgame
main_url= 'https://www.rottentomatoes.com/m/'
links = []
for movie in formatted:
    link = f'{main_url}{movie}'
    links.append(link)

# Loop for all pages     
for idx, link in enumerate(links):    
    r = requests.get(link)
    r.status_code
    if r.status_code != 200:
        try_again.append(movies[idx])
        continue
    try:    
        soup = BeautifulSoup(r.text, 'html.parser')

        # Title
        title = movies[idx]

        # RT rating
        rt_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[0].text

        # RT Cleaning
        rt_rating = rt_rating.replace("  ", '')
        rt_rating = rt_rating.replace("\n", '')

        # Audience rating
        aud_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[1].text

        # Audience Cleaning
        aud_rating = aud_rating.replace("  ", '')
        aud_rating = aud_rating.replace("\n", '')

        # Director 
        director = soup.find_all('li', class_="meta-row clearfix")[2].text

        # Director Cleaning
        director = director.replace("  ", '')
        director = director.replace("\n", '')
        director = director.replace("Directed By:", '')
        director = director.split(',')

        # Writers
        writers = soup.find_all('li', class_="meta-row clearfix")[3].text

        # Writers Cleaning
        writers = writers.replace("  ", '')
        writers = writers.replace("\n", '')
        writers = writers.replace("Written By:", '')
        writers = writers.split(',')

        # Top 5 billed Actors/Actresses
        test_ = soup.find_all('div', class_="castSection")[0].text
        test = test_.split("  \n  ")
        names = []
        for name in test:
            name = name.replace('\nas\n ', '')
            name = name.replace('\n', '')
            name = name.replace('  ', '')
            names.append(name)
        act = names[0:13:3]

        # Add to Mongo db
        rt_ratings.insert_one({'title': title, 'rotten_tomato_rating':rt_rating, 'audience_rating':aud_rating, 'director':director, 'writers':writers, 'top_5_billed':act})
    except:
        try_again.append(movies[idx])
    
    
    # Mongo print for sanity
    print(db.rt_ratings.find().count())
    
    # Add time before next request 
    slp = random.randint(5,15)
    time.sleep(slp)

/Users/Michael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


1
2
3
4
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
21
22
23
24
25
26
27
28
28
29
30
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
96
97
98
99
100
101
102
103
103
104
105
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
171
172
173
174
175
176
177
178
178
179
180
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
246
247
248
249
250
251
252
253
253
254
255
255
256
257
258
259
260


ConnectionError: HTTPSConnectionPool(host='www.rottentomatoes.com', port=443): Max retries exceeded with url: /m/Star_Trek_Beyond (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7feef95b2e10>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [136]:
#Debug area
print(soup.find_all('h2', class_="mop-ratings-wrap__score"))

[]


In [132]:
print(soup)

<!DOCTYPE html>

<html dir="ltr" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<!-- salt=lay-def-02-juRm -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Rotten Tomatoes: Movies - Rotten Tomatoes</title>
<meta content="Rotten Tomatoes, home of the Tomatometer, is the most trusted measurement of quality for Movies &amp; TV. The definitive site for Reviews, Trailers, Showtimes, and Tickets" name="description"/>
<link href="https://www.rottentomatoes.com/m/Star_Wars_Ep._VII_The_Force_Awakens" rel="canonical"/>
<link href="https://www.rottentomatoes.com/assets/pizza-pie/images/favicon.ico" rel="shortcut icon" sizes="76x76" type="image/x-icon"/>
<meta content="326803741017" property="fb

In [141]:
rt_pd = pd.DataFrame(list(rt_ratings.find()))

In [142]:
rt_pd.to_csv('data/rt_pd.csv', index=False)

In [ ]:
print(try_again)